In [60]:
import numpy as np
import pandas as pd
from unidecode import unidecode

import pyreadr

In [61]:
#double check matching with Kathryn's codes, directorio file in Box 6 digits, 7 digits, and TSE codes


#cd _municipio and id_municipio_tse (will comne with raw tse data)-- matching on name doesn't make sense since there can be multiple with same name
#prefeito only, extraodrinary elections, match on numbers

#id municpio matches with the vote muni ids (7 digits codes)

In [62]:
#this file contains municipality names and their codes. 
#the 2016 data has names but no codes!

#convert_muni = pd.read_csv('https://raw.githubusercontent.com/ekatovich/Municipality_Name_ID_crosswalk/master/Raw_Brazil_GeographicalUnits.csv', encoding='latin-1')
#convert_muni = convert_muni[['uf', 'uf_name', 'mesoregion_stub', 'mesoregion_name',
#       'microregion_stub', 'microregion_name', 'munic_stub', 'munic_code',
#       'munic_name']]
#convert_muni.munic_code = convert_muni.munic_code.astype(str)
#convert_muni.munic_name = convert_muni.munic_name.apply(unidecode).apply(lambda x: x.lower())
#convert_muni.munic_name = convert_muni.munic_name.apply(lambda x: x.replace('\'', '').replace(' ', ''))
#convert_muni.head()

In [63]:
df = pd.read_csv('/Users/annieulichney/Downloads/votacao_candidato_munzona_2016-2/votacao_candidato_munzona_2016_BRASIL.csv',  delimiter=';', encoding='latin1')
cols = ['ANO_ELEICAO', 'NR_TURNO', 'NM_TIPO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO', 'DS_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_UE', 'DS_SIT_TOT_TURNO', 'QT_VOTOS_NOMINAIS', 'SG_PARTIDO']
df = df[cols]

df.NM_TIPO_ELEICAO = df.NM_TIPO_ELEICAO.apply(unidecode).apply(lambda x: x.lower())
df.DS_CARGO = df.DS_CARGO.apply(unidecode).apply(lambda x: x.lower())
df.NM_UE = df.NM_UE.apply(unidecode).apply(lambda x: x.lower())
df.DS_SIT_TOT_TURNO = df.DS_SIT_TOT_TURNO.apply(unidecode).apply(lambda x: x.lower())
df.NM_UE = df.NM_UE.apply(lambda x: x.replace('\'', '').replace(' ', ''))

df = df[df.DS_CARGO == 'prefeito']

df = df[df.NM_TIPO_ELEICAO == 'eleicao ordinaria']

In [64]:
df.head()

,ANO_ELEICAO,NR_TURNO,NM_TIPO_ELEICAO,SG_UF,CD_MUNICIPIO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_UE,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,SG_PARTIDO
0,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002509,45,acrelandia,nao eleito,627,PSDB
1,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002445,40,acrelandia,eleito,3852,PSB
2,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001075,11,acrelandia,nao eleito,1545,PP
3,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002100,10,acrelandia,nao eleito,259,PRB
4,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001284,12,acrelandia,nao eleito,886,PDT


In [65]:
#import code directory to convert codigo municipios (cd_municipio --> id municipio_tse)

path = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/diretorio.Rdata'
diretorio = pyreadr.read_r(path)['df.diretorio']

diretorio = diretorio[['id_municipio_tse', 'id_municipio']]

df_merged = pd.merge(df, diretorio, how = 'left', left_on = 'CD_MUNICIPIO', right_on = 'id_municipio_tse')

df_merged['id_candidato_bd'] = np.nan

df_merged = df_merged [['ANO_ELEICAO', 'NR_TURNO', 'NM_TIPO_ELEICAO', 'SG_UF', 'id_municipio', 'id_municipio_tse', 'DS_CARGO', 
                    'SG_PARTIDO', 'NR_CANDIDATO', 'SQ_CANDIDATO', 'id_candidato_bd', 'DS_SIT_TOT_TURNO', 'QT_VOTOS_NOMINAIS']]

In [66]:
df_merged.columns= ['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd', 'resultado', 'votos']

In [67]:
df_merged.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,2016,1,eleicao ordinaria,AC,1200013,1120,prefeito,PSDB,45,10000002509,NaN,nao eleito,627
1,2016,1,eleicao ordinaria,AC,1200013,1120,prefeito,PSB,40,10000002445,NaN,eleito,3852
2,2016,1,eleicao ordinaria,AC,1200013,1120,prefeito,PP,11,10000001075,NaN,nao eleito,1545
3,2016,1,eleicao ordinaria,AC,1200013,1120,prefeito,PRB,10,10000002100,NaN,nao eleito,259
4,2016,1,eleicao ordinaria,AC,1200013,1120,prefeito,PDT,12,10000001284,NaN,nao eleito,886


In [70]:
df_grouped = df_merged.groupby(['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio', 'cargo', 'numero_candidato', 'sequencial_candidato', 'resultado', 'sigla_partido'], as_index = False).sum()

In [87]:
#df_merged.to_csv('election_results_2016.csv', index = False)

df_grouped.to_csv('election_results_2016.csv', index = False)

In [86]:
#here's a check of our merge

df_grouped[(df_grouped.id_municipio == 3548708) & (df_grouped.numero_candidato == 23) & (df_grouped.turno == 2)].votos.sum() == df_merged[(df_merged.id_municipio == 3548708)&(df_merged.turno == 2) & (df_merged.numero_candidato == 23)].votos.sum()

True

In [90]:
df_grouped[df_grouped.id_municipio == 3551306]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,cargo,numero_candidato,sequencial_candidato,resultado,sigla_partido,id_municipio_tse,id_candidato_bd,votos
14969,2016,1,eleicao ordinaria,SP,3551306,prefeito,40,250000033832,nao eleito,PSB,71277,0.0,1083


In [106]:
no_winners = []
for muni in np.unique(df.CD_MUNICIPIO):
    if df[(df.CD_MUNICIPIO == muni)&(df.DS_SIT_TOT_TURNO == 'eleito')].shape[0] == 0:
        no_winners.append(muni)

In [107]:
no_winners

[31291,
 31712,
 40479,
 40592,
 40665,
 41483,
 41807,
 42188,
 42277,
 42331,
 43532,
 44075,
 44792,
 45594,
 45993,
 46256,
 47295,
 51659,
 52159,
 53732,
 56758,
 62030,
 62855,
 64190,
 64394,
 64459,
 64831,
 66273,
 66931,
 67032,
 67113,
 68837,
 70114,
 70270,
 71277,
 72079,
 75850,
 80012,
 81000,
 83895,
 85278,
 85383,
 85537,
 86754,
 86835,
 89753,
 94897]

In [99]:
df_merged

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,2016,1,eleicao ordinaria,AC,1200013,1120,prefeito,PSDB,45,10000002509,NaN,nao eleito,627
1,2016,1,eleicao ordinaria,AC,1200013,1120,prefeito,PSB,40,10000002445,NaN,eleito,3852
2,2016,1,eleicao ordinaria,AC,1200013,1120,prefeito,PP,11,10000001075,NaN,nao eleito,1545
3,2016,1,eleicao ordinaria,AC,1200013,1120,prefeito,PRB,10,10000002100,NaN,nao eleito,259
4,2016,1,eleicao ordinaria,AC,1200013,1120,prefeito,PDT,12,10000001284,NaN,nao eleito,886
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21606,2016,2,eleicao ordinaria,SP,3548708,70750,prefeito,PSDB,45,250000010056,NaN,eleito,34691
21607,2016,2,eleicao ordinaria,SP,3548708,70750,prefeito,PPS,23,250000013230,NaN,nao eleito,34438
21608,2016,2,eleicao ordinaria,SP,3548708,70750,prefeito,PSDB,45,250000010056,NaN,eleito,42241
21609,2016,2,eleicao ordinaria,SP,3548708,70750,prefeito,PPS,23,250000013230,NaN,nao eleito,17126


In [19]:
diretorio[diretorio.id_municipio_bcb == '1139']

,id_municipio,id_municipio_6,id_municipio_tse,id_municipio_rf,id_municipio_bcb,municipio,capital_uf,id_comarca,id_regiao_saude,regiao_saude,...,microrregiao,id_mesorregiao,mesorregiao,id_uf,sigla_uf,uf,regiao,existia_1991,existia_2000,existia_2010


In [20]:
np.unique(df.CD_MUNICIPIO)

array([   19,    35,    51, ..., 99031, 99058, 99074])

In [23]:
np.unique(diretorio.id_municipio_tse)

array([   19,    35,    51, ..., 99031, 99058, 99074], dtype=int32)

In [15]:
df.head(30)

,ANO_ELEICAO,NR_TURNO,NM_TIPO_ELEICAO,SG_UF,CD_MUNICIPIO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_UE,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,SG_PARTIDO
0,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002509,45,acrelandia,nao eleito,627,PSDB
1,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002445,40,acrelandia,eleito,3852,PSB
2,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001075,11,acrelandia,nao eleito,1545,PP
3,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002100,10,acrelandia,nao eleito,259,PRB
4,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001284,12,acrelandia,nao eleito,886,PDT
92,2016,1,eleicao ordinaria,AC,1570,prefeito,10000002636,65,assisbrasil,nao eleito,1342,PC do B
93,2016,1,eleicao ordinaria,AC,1570,prefeito,10000003037,25,assisbrasil,nao eleito,753,DEM
94,2016,1,eleicao ordinaria,AC,1570,prefeito,10000001848,45,assisbrasil,eleito,2324,PSDB
144,2016,1,eleicao ordinaria,AC,1058,prefeito,10000001187,13,brasileia,eleito,6819,PT
145,2016,1,eleicao ordinaria,AC,1058,prefeito,10000002360,45,brasileia,nao eleito,6013,PSDB


In [8]:
df_merged

,ANO_ELEICAO,NR_TURNO,NM_TIPO_ELEICAO,SG_UF,CD_MUNICIPIO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_UE,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,SG_PARTIDO,id_municipio_tse,id_municipio_bcb,municipio
0,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002509,45,acrelandia,nao eleito,627,PSDB,NaN,NaN,NaN
1,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002445,40,acrelandia,eleito,3852,PSB,NaN,NaN,NaN
2,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001075,11,acrelandia,nao eleito,1545,PP,NaN,NaN,NaN
3,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002100,10,acrelandia,nao eleito,259,PRB,NaN,NaN,NaN
4,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001284,12,acrelandia,nao eleito,886,PDT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21606,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,34691,PSDB,NaN,NaN,NaN
21607,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,34438,PPS,NaN,NaN,NaN
21608,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,42241,PSDB,NaN,NaN,NaN
21609,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,17126,PPS,NaN,NaN,NaN


In [15]:
diretorio

,id_municipio,id_municipio_6,id_municipio_tse,id_municipio_rf,id_municipio_bcb,municipio,capital_uf,id_comarca,id_regiao_saude,regiao_saude,...,microrregiao,id_mesorregiao,mesorregiao,id_uf,sigla_uf,uf,regiao,existia_1991,existia_2000,existia_2010
0,1100023,110002,78,7,9393,Ariquemes,0,1100023,11001,Vale do Jamari,...,Ariquemes,1102,Leste Rondoniense,11,RO,Rondônia,Norte,1,1,1
1,1100106,110010,19,1,22882,Guajará-Mirim,0,1100106,11004,Madeira-Mamoré,...,Guajará-Mirim,1101,Madeira-Guaporé,11,RO,Rondônia,Norte,1,1,1
2,1100114,110011,159,15,41258,Jaru,0,1100114,11003,Central,...,Ji-Paraná,1102,Leste Rondoniense,11,RO,Rondônia,Norte,1,1,1
3,1100130,110013,396,39,44547,Machadinho D'Oeste,0,1100122,11001,Vale do Jamari,...,Ariquemes,1102,Leste Rondoniense,11,RO,Rondônia,Norte,1,1,1
4,1100205,110020,35,3,30719,Porto Velho,1,1100205,11004,Madeira-Mamoré,...,Porto Velho,1101,Madeira-Guaporé,11,RO,Rondônia,Norte,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,2311801,231180,15377,1537,9843,Russas,0,2311801,23009,9ª Região Russas,...,Baixo Jaguaribe,2305,Jaguaribe,23,CE,Ceará,Nordeste,1,1,1
5566,2312502,231250,15512,1551,36548,São João do Jaguaribe,0,2313104,23010,10ª Região Limoeiro do Norte,...,Baixo Jaguaribe,2305,Jaguaribe,23,CE,Ceará,Nordeste,1,1,1
5567,2312700,231270,15555,1555,2462,Senador Pompeu,0,2312700,23008,8ª Região Quixadá,...,Sertão de Senador Pompeu,2304,Sertões Cearenses,23,CE,Ceará,Nordeste,1,1,1
5568,2313005,231300,15610,1561,34818,Solonópole,0,2313005,23008,8ª Região Quixadá,...,Sertão de Senador Pompeu,2304,Sertões Cearenses,23,CE,Ceará,Nordeste,1,1,1


In [11]:
diretorio.head()

,id_municipio,id_municipio_6,id_municipio_tse,id_municipio_rf,id_municipio_bcb,municipio,capital_uf,id_comarca,id_regiao_saude,regiao_saude,...,microrregiao,id_mesorregiao,mesorregiao,id_uf,sigla_uf,uf,regiao,existia_1991,existia_2000,existia_2010
0,1100023,110002,78,7,9393,Ariquemes,0,1100023,11001,Vale do Jamari,...,Ariquemes,1102,Leste Rondoniense,11,RO,Rondônia,Norte,1,1,1
1,1100106,110010,19,1,22882,Guajará-Mirim,0,1100106,11004,Madeira-Mamoré,...,Guajará-Mirim,1101,Madeira-Guaporé,11,RO,Rondônia,Norte,1,1,1
2,1100114,110011,159,15,41258,Jaru,0,1100114,11003,Central,...,Ji-Paraná,1102,Leste Rondoniense,11,RO,Rondônia,Norte,1,1,1
3,1100130,110013,396,39,44547,Machadinho D'Oeste,0,1100122,11001,Vale do Jamari,...,Ariquemes,1102,Leste Rondoniense,11,RO,Rondônia,Norte,1,1,1
4,1100205,110020,35,3,30719,Porto Velho,1,1100205,11004,Madeira-Mamoré,...,Porto Velho,1101,Madeira-Guaporé,11,RO,Rondônia,Norte,1,1,1


In [37]:
df[(df.NM_UE == 'aguabranca')&(df.DS_SIT_TOT_TURNO == 'eleito')]

,ANO_ELEICAO,NR_TURNO,NM_TIPO_ELEICAO,SG_UF,CD_MUNICIPIO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_UE,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,SG_PARTIDO
10233,2016,1,eleicao ordinaria,AL,27014,prefeito,20000005941,45,aguabranca,eleito,6151,PSDB
363844,2016,1,eleicao ordinaria,PB,19011,prefeito,150000004710,15,aguabranca,eleito,2974,PMDB
409225,2016,1,eleicao ordinaria,PI,10030,prefeito,180000001182,55,aguabranca,eleito,6354,PSD


In [35]:
for mun_test in np.unique(df.NM_UE)[0:1000]:
    if (df[(df.NM_UE == mun_test)&(df.DS_SIT_TOT_TURNO == 'eleito')].shape[0]) != 1:
        print(mun_test, df[(df.NM_UE == mun_test)&(df.DS_SIT_TOT_TURNO == 'eleito')].shape[0])

        #print( np.unique(df[(df.NM_UE == mun_test)&(df.DS_SIT_TOT_TURNO == 'eleito')].NR_TURNO).shape[0], np.unique(df[(df.NM_UE == mun_test)&(df.DS_SIT_TOT_TURNO == 'eleito')].SG_UF).shape[0])
        #print()

abelardoluz 0
acailandia 2
aguaboa 2
aguabranca 3
alagoinha 2
alagoinhas 2
altoalegre 3
altoparaiso 2
alvorada 3
alvoradademinas 0
americana 2
amparo 2
ananindeua 2
anapolis 4
anchieta 2
angradosreis 2
antoniocarlos 2
antoniodias 0
aparecida 2
aparecidadegoiania 3
apucarana 2
aracaju 4
aracatuba 2
aracoiaba 2
araguaina 2
araguana 2
arapiraca 2
arapongas 2
arapua 2
araraquara 2
araruna 2
areiabranca 2
ariquemes 2
arvorezinha 0
assis 2
atalaia 2
aurora 2
bacabal 2
bage 2
balneariocamboriu 2
balsas 2
bandeirantes 2
barauna 2
bariri 0
barrabonita 2
barracao 2
barradesaomiguel 2
barradocorda 2
barradogarcas 2
barramansa 3
barreiras 2
barroalto 2
barueri 2
batalha 2
bauru 3
belem 13
belfordroxo 4
belmonte 2
belohorizonte 18
betim 3
blumenau 3
boaesperanca 3
boavista 3
bocaina 2
bomjardim 3
bomjardimdaserra 0
bomjesus 5
bomjesusdotocantins 2
bomsucesso 3
bonfim 2
bonito 4
borborema 2
brejinho 2
brusque 2
buritis 2
butia 0
cabodesantoagostinho 2
cabofrio 2
cachoeiradourada 2
cachoeirinha 4
cac

In [22]:
np.unique(df[(df.NM_UE == 'saosebastiao')&(df.DS_SIT_TOT_TURNO == 'eleito')].SG_UF).shape[0]

2

In [7]:
df = df[df.NM_TIPO_ELEICAO == 'eleicao ordinaria']


#there are a few duplicate entries. perhaps recounts? 
#talk to Kathryn about this and for now take means

##df = df.groupby(['ANO_ELEICAO', 'NR_TURNO', 'NM_TIPO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
#       'DS_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_UE', 'DS_SIT_TOT_TURNO',
#       'SG_PARTIDO'], as_index=False).mean()

In [5]:
#df[df.CD_MUNICIPIO == 19]

In [6]:
df.tail()

,ANO_ELEICAO,NR_TURNO,NM_TIPO_ELEICAO,SG_UF,CD_MUNICIPIO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_UE,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,SG_PARTIDO
940017,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,34691,PSDB
940018,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,34438,PPS
940019,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,42241,PSDB
940020,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,17126,PPS
940021,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,30476,PSDB


In [7]:
# some of the names in convert_muni are slightly different from those in df that we want to merge on

#change names in convert_muni to these:
replace = ['alvoradadooeste', 'assu', 'boasaude', 'camaca',
       'eldoradodoscarajas', 'espigaodooeste', 'gracchocardoso',
       'grao-para', 'olhosdagua', 'quinjingue', 'santaisabeldopara',
       'saoluisdoparaitinga', 'sempeixe']

#in convert muni they originally exist as:
original = ['alvoradadoeste', 'acu', 'januariocicco', 'camacan',
       'eldoradodocarajas', 'espigaodoeste', 'grachocardoso',
       'graopara', 'olhos-dagua', 'quijingue', 'santaizabeldopara',
       'saoluizdoparaitinga', 'sem-peixe']


In [8]:

for i in range(len(replace)):
    convert_muni.loc[convert_muni[convert_muni.munic_name == original[i]].index, 'munic_name'] = replace[i]

In [9]:
df.head()

,ANO_ELEICAO,NR_TURNO,NM_TIPO_ELEICAO,SG_UF,CD_MUNICIPIO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_UE,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,SG_PARTIDO
0,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002509,45,acrelandia,nao eleito,627,PSDB
1,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002445,40,acrelandia,eleito,3852,PSB
2,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001075,11,acrelandia,nao eleito,1545,PP
3,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002100,10,acrelandia,nao eleito,259,PRB
4,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001284,12,acrelandia,nao eleito,886,PDT


In [10]:
convert_muni.head()

,uf,uf_name,mesoregion_stub,mesoregion_name,microregion_stub,microregion_name,munic_stub,munic_code,munic_name
0,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,altaflorestadoeste
1,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,altoalegredosparecis
2,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,altoparaiso
3,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,alvoradadooeste
4,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,ariquemes


In [11]:
df_merged = pd.merge(df, convert_muni[['munic_code', 'munic_name']], how = 'left', left_on = 'NM_UE', right_on = 'munic_name')

In [12]:
np.unique(df_merged[df_merged.munic_code.isna()].NM_UE)

array([], dtype=object)

In [14]:
#df_merged['id_municipio_tse'] = np.nan
#df_merged['id_candidato_bd'] = np.nan

#df_merged.columns = ['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio_old',
#       'cargo', 'sequencial_candidato', 'numero_candidato', 'munic_name1', 'resultado',
#       'votos', 'sigla_partido', 'id_municipio', 'munic_name',  'id_municipio_tse', 'id_candidato_bd']

#df_merged = df_merged[['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
#       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
#       'sequencial_candidato', 'id_candidato_bd', 'resultado', 'votos']]



df_merged.columns = ['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio_old',
      'cargo', 'sequencial_candidato', 'numero_candidato', 'munic_name1', 'resultado',
       'votos', 'sigla_partido', 'id_municipio', 'munic_name']

df_merged = df_merged[['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'resultado', 'votos']]

In [31]:
df_merged.tail(14)

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,cargo,sigla_partido,numero_candidato,sequencial_candidato,resultado,votos
1024897,2016,2,eleicao ordinaria,SP,3552502,prefeito,PTB,14,250000023842,nao eleito,26426
1024898,2016,2,eleicao ordinaria,SP,3552502,prefeito,PR,22,250000035239,eleito,37603
1024899,2016,2,eleicao ordinaria,SP,3548708,prefeito,PPS,23,250000013230,nao eleito,35225
1024900,2016,2,eleicao ordinaria,SP,3548708,prefeito,PSDB,45,250000010056,eleito,46605
1024901,2016,2,eleicao ordinaria,SP,3548708,prefeito,PPS,23,250000013230,nao eleito,14515
1024902,2016,2,eleicao ordinaria,SP,3548708,prefeito,PSDB,45,250000010056,eleito,27942
1024903,2016,2,eleicao ordinaria,SP,3548708,prefeito,PPS,23,250000013230,nao eleito,15363
1024904,2016,2,eleicao ordinaria,SP,3548708,prefeito,PSDB,45,250000010056,eleito,31706
1024905,2016,2,eleicao ordinaria,SP,3548708,prefeito,PPS,23,250000013230,nao eleito,26150
1024906,2016,2,eleicao ordinaria,SP,3548708,prefeito,PSDB,45,250000010056,eleito,34691


In [45]:
df_grouped = df_merged.groupby(['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio', 'cargo', 'numero_candidato', 'sequencial_candidato', 'resultado', 'sigla_partido'], as_index = False).mean()

In [46]:
df_grouped['id_municipio_tse'] = np.nan
df_grouped['id_candidato_bd'] = np.nan

In [47]:
df_grouped.tail()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,cargo,numero_candidato,sequencial_candidato,resultado,sigla_partido,votos,id_municipio_tse,id_candidato_bd
500838,2016,2,eleicao ordinaria,SP,3548708,prefeito,45,250000010056,eleito,PSDB,35610.166667,NaN,NaN
500839,2016,2,eleicao ordinaria,SP,3552205,prefeito,25,250000005826,eleito,DEM,30472.166667,NaN,NaN
500840,2016,2,eleicao ordinaria,SP,3552205,prefeito,50,250000004952,nao eleito,PSOL,21630.666667,NaN,NaN
500841,2016,2,eleicao ordinaria,SP,3552502,prefeito,14,250000023842,nao eleito,PTB,28093.000000,NaN,NaN
500842,2016,2,eleicao ordinaria,SP,3552502,prefeito,22,250000035239,eleito,PR,41456.000000,NaN,NaN


In [30]:
#check for this merge

#df_grouped[df_grouped.id_municipio == '3548708']
#df_merged[(df_merged.turno == 2) & (df_merged.id_municipio == '3548708') & (df_merged.numero_candidato == 23)].votos.mean()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,cargo,numero_candidato,sequencial_candidato,resultado,votos
475520,2016,1,eleicao ordinaria,SP,3548708,prefeito,13,250000034223,nao eleito,14128.000000
475521,2016,1,eleicao ordinaria,SP,3548708,prefeito,15,250000077933,nao eleito,1174.333333
475522,2016,1,eleicao ordinaria,SP,3548708,prefeito,16,250000067214,nao eleito,135.833333
475523,2016,1,eleicao ordinaria,SP,3548708,prefeito,23,250000013230,2o turno,17787.666667
475524,2016,1,eleicao ordinaria,SP,3548708,prefeito,45,250000010056,2o turno,28218.333333
...,...,...,...,...,...,...,...,...,...,...
476230,2016,1,eleicao ordinaria,SP,3548708,vereador,90737,250000050924,suplente,15.166667
476231,2016,1,eleicao ordinaria,SP,3548708,vereador,90777,250000050927,suplente,52.833333
476232,2016,1,eleicao ordinaria,SP,3548708,vereador,90900,250000050929,suplente,99.666667
500837,2016,2,eleicao ordinaria,SP,3548708,prefeito,23,250000013230,nao eleito,23802.833333


In [48]:
df_merged.shape[0]

1024911

In [49]:
df_grouped.to_csv('election_results_2016.csv', index = False)

In [50]:
df_grouped.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,cargo,numero_candidato,sequencial_candidato,resultado,sigla_partido,votos,id_municipio_tse,id_candidato_bd
0,2016,1,eleicao ordinaria,AC,1200013,prefeito,10,10000002100,nao eleito,PRB,259.0,NaN,NaN
1,2016,1,eleicao ordinaria,AC,1200013,prefeito,11,10000001075,nao eleito,PP,1545.0,NaN,NaN
2,2016,1,eleicao ordinaria,AC,1200013,prefeito,12,10000001284,nao eleito,PDT,886.0,NaN,NaN
3,2016,1,eleicao ordinaria,AC,1200013,prefeito,40,10000002445,eleito,PSB,3852.0,NaN,NaN
4,2016,1,eleicao ordinaria,AC,1200013,prefeito,45,10000002509,nao eleito,PSDB,627.0,NaN,NaN


In [51]:
df_grouped[(df_grouped.ano == 2016)&(df_grouped.id_municipio == '1200401')]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,cargo,numero_candidato,sequencial_candidato,resultado,sigla_partido,votos,id_municipio_tse,id_candidato_bd
1159,2016,1,eleicao ordinaria,AC,1200401,prefeito,13,10000000922,eleito,PT,34770.333333,NaN,NaN
1160,2016,1,eleicao ordinaria,AC,1200401,prefeito,15,10000001231,nao eleito,PMDB,20290.333333,NaN,NaN
1161,2016,1,eleicao ordinaria,AC,1200401,prefeito,18,10000002300,nao eleito,REDE,5245.000000,NaN,NaN
1162,2016,1,eleicao ordinaria,AC,1200401,prefeito,22,10000002141,nao eleito,PR,3067.333333,NaN,NaN
1163,2016,1,eleicao ordinaria,AC,1200401,vereador,10000,10000001033,suplente,PRB,139.666667,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209424,2016,1,eleicao ordinaria,MT,1200401,vereador,55678,110000004834,suplente,PSD,0.000000,NaN,NaN
209425,2016,1,eleicao ordinaria,MT,1200401,vereador,55888,110000004837,suplente,PSD,44.000000,NaN,NaN
209426,2016,1,eleicao ordinaria,MT,1200401,vereador,90000,110000004832,eleito por qp,PROS,207.000000,NaN,NaN
209427,2016,1,eleicao ordinaria,MT,1200401,vereador,90222,110000004833,eleito por qp,PROS,244.000000,NaN,NaN


In [35]:
df_merged.groupby(['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd', 'resultado'], as_index = False).mean()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos


In [33]:
df_merged.votos.mean()

242.7921478060046

In [38]:
df_merged.dtypes

ano                       int64
turno                     int64
tipo_eleicao             object
sigla_uf                 object
id_municipio             object
id_municipio_tse        float64
cargo                    object
sigla_partido            object
numero_candidato          int64
sequencial_candidato      int64
id_candidato_bd         float64
resultado                object
votos                     int64
dtype: object

In [16]:
df_merged.columns = ['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio_old',
       'cargo', 'sequencial_candidato', 'numero_candidato', 'munic_name1', 'resultado',
       'sigla_partido', 'votos', 'id_municipio', 'munic_name',  'id_municipio_tse', 'id_candidato_bd']

In [290]:
df_merged = df_merged[['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd', 'resultado', 'votos']]

In [291]:
df_merged

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,2016,1,eleicao ordinaria,AC,1200013,NaN,prefeito,PSDB,45,10000002509,NaN,nao eleito,627
1,2016,1,eleicao ordinaria,AC,1200013,NaN,prefeito,PSB,40,10000002445,NaN,eleito,3852
2,2016,1,eleicao ordinaria,AC,1200013,NaN,prefeito,PP,11,10000001075,NaN,nao eleito,1545
3,2016,1,eleicao ordinaria,AC,1200013,NaN,prefeito,PRB,10,10000002100,NaN,nao eleito,259
4,2016,1,eleicao ordinaria,AC,1200013,NaN,prefeito,PDT,12,10000001284,NaN,nao eleito,886
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024906,2016,2,eleicao ordinaria,SP,3548708,NaN,prefeito,PSDB,45,250000010056,NaN,eleito,34691
1024907,2016,2,eleicao ordinaria,SP,3548708,NaN,prefeito,PPS,23,250000013230,NaN,nao eleito,34438
1024908,2016,2,eleicao ordinaria,SP,3548708,NaN,prefeito,PSDB,45,250000010056,NaN,eleito,42241
1024909,2016,2,eleicao ordinaria,SP,3548708,NaN,prefeito,PPS,23,250000013230,NaN,nao eleito,17126


In [297]:
df_merged.groupby(['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd', 'resultado'], as_index = False)

In [292]:
df_merged.groupby(['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio',
       'id_municipio_tse', 'cargo', 'sigla_partido', 'numero_candidato',
       'sequencial_candidato', 'id_candidato_bd', 'resultado'], as_index = False).mean()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos


In [234]:
df_merged.groupby(['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'cargo',
       'sequencial_candidato', 'numero_candidato', 'munic_name', 'resultado',
        'sigla_partido', 'id_municipio', 'id_municipio_tse',
       'id_candidato_bd'], as_index = False).mean()

KeyError: 'id_municipio_old'

In [209]:
#df_merged = df_merged.loc[:,~df_merged.columns.duplicated()]

In [210]:
df_merged

,ano,turno,tipo_eleicao,sigla_uf,id_municipio_old,cargo,sequencial_candidato,numero_candidato,munic_name,resultado,sigla_partido,votos,id_municipio,id_municipio_tse,id_candidato_bd
0,2016,1,eleicao ordinaria,AC,1007,prefeito,10000001880,65,bujari,eleito,PC do B,1853.000000,1200138,NaN,NaN
1,2016,1,eleicao ordinaria,AC,1007,prefeito,10000002320,13,bujari,nao eleito,PT,1145.000000,1200138,NaN,NaN
2,2016,1,eleicao ordinaria,AC,1007,prefeito,10000002447,15,bujari,nao eleito,PMDB,1038.000000,1200138,NaN,NaN
3,2016,1,eleicao ordinaria,AC,1007,prefeito,10000002574,11,bujari,nao eleito,PP,1468.000000,1200138,NaN,NaN
4,2016,1,eleicao ordinaria,AC,1007,prefeito,10000003549,40,bujari,nao eleito,PSB,700.000000,1200138,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500838,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,PPS,23802.833333,3548708,NaN,NaN
500839,2016,2,eleicao ordinaria,SP,71455,prefeito,250000004952,50,sorocaba,nao eleito,PSOL,21630.666667,3552205,NaN,NaN
500840,2016,2,eleicao ordinaria,SP,71455,prefeito,250000005826,25,sorocaba,eleito,DEM,30472.166667,3552205,NaN,NaN
500841,2016,2,eleicao ordinaria,SP,71510,prefeito,250000023842,14,suzano,nao eleito,PTB,28093.000000,3552502,NaN,NaN


In [211]:
df_merged = df_merged.loc[:,~df_merged.columns.duplicated()].copy()

In [212]:
df_merged

,ano,turno,tipo_eleicao,sigla_uf,id_municipio_old,cargo,sequencial_candidato,numero_candidato,munic_name,resultado,sigla_partido,votos,id_municipio,id_municipio_tse,id_candidato_bd
0,2016,1,eleicao ordinaria,AC,1007,prefeito,10000001880,65,bujari,eleito,PC do B,1853.000000,1200138,NaN,NaN
1,2016,1,eleicao ordinaria,AC,1007,prefeito,10000002320,13,bujari,nao eleito,PT,1145.000000,1200138,NaN,NaN
2,2016,1,eleicao ordinaria,AC,1007,prefeito,10000002447,15,bujari,nao eleito,PMDB,1038.000000,1200138,NaN,NaN
3,2016,1,eleicao ordinaria,AC,1007,prefeito,10000002574,11,bujari,nao eleito,PP,1468.000000,1200138,NaN,NaN
4,2016,1,eleicao ordinaria,AC,1007,prefeito,10000003549,40,bujari,nao eleito,PSB,700.000000,1200138,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500838,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,PPS,23802.833333,3548708,NaN,NaN
500839,2016,2,eleicao ordinaria,SP,71455,prefeito,250000004952,50,sorocaba,nao eleito,PSOL,21630.666667,3552205,NaN,NaN
500840,2016,2,eleicao ordinaria,SP,71455,prefeito,250000005826,25,sorocaba,eleito,DEM,30472.166667,3552205,NaN,NaN
500841,2016,2,eleicao ordinaria,SP,71510,prefeito,250000023842,14,suzano,nao eleito,PTB,28093.000000,3552502,NaN,NaN


In [213]:
df_merged.groupby(['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio_old', 'cargo',
       'sequencial_candidato', 'numero_candidato', 'munic_name', 'resultado',
        'sigla_partido', 'id_municipio', 'id_municipio_tse',
       'id_candidato_bd'], as_index = False).mean()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio_old,cargo,sequencial_candidato,numero_candidato,munic_name,resultado,sigla_partido,id_municipio,id_municipio_tse,id_candidato_bd,votos


In [114]:
df_merged.to_csv('election_results_2016.csv', index = False)

In [135]:
df_merged.columns

Index(['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio_old', 'cargo',
       'sequencial_candidato', 'numero_candidato', 'munic_name', 'resultado',
       'votos', 'sigla_partido', 'id_municipio', 'id_municipio_tse',
       'id_candidato_bd'],
      dtype='object')

In [128]:
df_merged.munic_name

,munic_name,munic_name
0,acrelandia,acrelandia
1,acrelandia,acrelandia
2,acrelandia,acrelandia
3,acrelandia,acrelandia
4,acrelandia,acrelandia
...,...,...
1024906,saobernardodocampo,saobernardodocampo
1024907,saobernardodocampo,saobernardodocampo
1024908,saobernardodocampo,saobernardodocampo
1024909,saobernardodocampo,saobernardodocampo


In [136]:
df_merged.groupby(['ano', 'turno', 'tipo_eleicao', 'sigla_uf', 'id_municipio_old', 'cargo',
       'sequencial_candidato', 'numero_candidato', 'munic_name', 'resultado',
        'sigla_partido', 'id_municipio', 'id_municipio_tse',
       'id_candidato_bd'], as_index = False).mean()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio_old,cargo,sequencial_candidato,numero_candidato,munic_name,resultado,sigla_partido,id_municipio,id_municipio_tse,id_candidato_bd,votos


In [122]:
df_merged.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio_old,cargo,sequencial_candidato,numero_candidato,munic_name,resultado,votos,sigla_partido,id_municipio,munic_name,id_municipio_tse,id_candidato_bd
0,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002509,45,acrelandia,nao eleito,627,PSDB,1200013,acrelandia,NaN,NaN
1,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002445,40,acrelandia,eleito,3852,PSB,1200013,acrelandia,NaN,NaN
2,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001075,11,acrelandia,nao eleito,1545,PP,1200013,acrelandia,NaN,NaN
3,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002100,10,acrelandia,nao eleito,259,PRB,1200013,acrelandia,NaN,NaN
4,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001284,12,acrelandia,nao eleito,886,PDT,1200013,acrelandia,NaN,NaN


In [119]:
df_merged.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio_old,cargo,sequencial_candidato,numero_candidato,munic_name,resultado,votos,sigla_partido,id_municipio,munic_name,id_municipio_tse,id_candidato_bd
0,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002509,45,acrelandia,nao eleito,627,PSDB,1200013,acrelandia,NaN,NaN
1,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002445,40,acrelandia,eleito,3852,PSB,1200013,acrelandia,NaN,NaN
2,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001075,11,acrelandia,nao eleito,1545,PP,1200013,acrelandia,NaN,NaN
3,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002100,10,acrelandia,nao eleito,259,PRB,1200013,acrelandia,NaN,NaN
4,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001284,12,acrelandia,nao eleito,886,PDT,1200013,acrelandia,NaN,NaN


In [116]:
df = df.groupby(['ANO_ELEICAO', 'NR_TURNO', 'NM_TIPO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'DS_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_UE', 'DS_SIT_TOT_TURNO',
       'SG_PARTIDO'], as_index=False).mean()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio_old,cargo,sequencial_candidato,numero_candidato,munic_name,resultado,votos,sigla_partido,id_municipio,munic_name,id_municipio_tse,id_candidato_bd
0,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002509,45,acrelandia,nao eleito,627,PSDB,1200013,acrelandia,NaN,NaN
1,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002445,40,acrelandia,eleito,3852,PSB,1200013,acrelandia,NaN,NaN
2,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001075,11,acrelandia,nao eleito,1545,PP,1200013,acrelandia,NaN,NaN
3,2016,1,eleicao ordinaria,AC,1120,prefeito,10000002100,10,acrelandia,nao eleito,259,PRB,1200013,acrelandia,NaN,NaN
4,2016,1,eleicao ordinaria,AC,1120,prefeito,10000001284,12,acrelandia,nao eleito,886,PDT,1200013,acrelandia,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024906,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,34691,PSDB,3548708,saobernardodocampo,NaN,NaN
1024907,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,34438,PPS,3548708,saobernardodocampo,NaN,NaN
1024908,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,42241,PSDB,3548708,saobernardodocampo,NaN,NaN
1024909,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,17126,PPS,3548708,saobernardodocampo,NaN,NaN


In [81]:
df.tail(10)

,ANO_ELEICAO,NR_TURNO,NM_TIPO_ELEICAO,SG_UF,CD_MUNICIPIO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_UE,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS,SG_PARTIDO
940012,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,14515,PPS
940013,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,27942,PSDB
940014,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,15363,PPS
940015,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,31706,PSDB
940016,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,26150,PPS
940017,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,34691,PSDB
940018,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,34438,PPS
940019,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,42241,PSDB
940020,2016,2,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,nao eleito,17126,PPS
940021,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,30476,PSDB


In [87]:
np.mean(df[(df.CD_MUNICIPIO == 70750) & (df.NR_TURNO == 2) & (df.NR_CANDIDATO == 45)].QT_VOTOS_NOMINAIS)

35610.166666666664

In [75]:
df.columns

Index(['ANO_ELEICAO', 'NR_TURNO', 'NM_TIPO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'DS_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_UE', 'DS_SIT_TOT_TURNO',
       'QT_VOTOS_NOMINAIS', 'SG_PARTIDO'],
      dtype='object')

In [78]:
df = df.groupby(['ANO_ELEICAO', 'NR_TURNO', 'NM_TIPO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'DS_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_UE', 'DS_SIT_TOT_TURNO',
       'SG_PARTIDO'], as_index=False).mean()

In [80]:
temp_df[temp_df.CD_MUNICIPIO == 70750]

,ANO_ELEICAO,NR_TURNO,NM_TIPO_ELEICAO,SG_UF,CD_MUNICIPIO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_UE,DS_SIT_TOT_TURNO,SG_PARTIDO,QT_VOTOS_NOMINAIS
428943,2016,1,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,2o turno,PSDB,28218.333333
428944,2016,1,eleicao ordinaria,SP,70750,prefeito,250000013230,23,saobernardodocampo,2o turno,PPS,17787.666667
428945,2016,1,eleicao ordinaria,SP,70750,prefeito,250000034223,13,saobernardodocampo,nao eleito,PT,14128.000000
428946,2016,1,eleicao ordinaria,SP,70750,prefeito,250000053011,50,saobernardodocampo,nao eleito,PSOL,1162.000000
428947,2016,1,eleicao ordinaria,SP,70750,prefeito,250000067214,16,saobernardodocampo,nao eleito,PSTU,135.833333
...,...,...,...,...,...,...,...,...,...,...,...,...
429653,2016,1,eleicao ordinaria,SP,70750,vereador,250000094106,40140,saobernardodocampo,nao eleito,PSB,5.000000
429654,2016,1,eleicao ordinaria,SP,70750,vereador,250000094470,13777,saobernardodocampo,suplente,PT,2.000000
429655,2016,1,eleicao ordinaria,SP,70750,vereador,250000094572,28001,saobernardodocampo,nao eleito,PRTB,10.333333
449000,2016,2,eleicao ordinaria,SP,70750,prefeito,250000010056,45,saobernardodocampo,eleito,PSDB,35610.166667
